# Toy model

This toy model has the purpose of testing the VEGAS algorithm in order to compute a discret sum.

I consider a function $f$ defined as:

$$
f(x_0, y_0, z_0) = \sum\limits_{x = 0}^{10} \sum\limits_{y = 0}^{10} \sum\limits_{z = 0}^{10} e^{-\frac{1}{2} \left( x - x_0 \right)^2} e^{-\frac{1}{2} \left( y - y_0 \right)^2} e^{-\frac{1}{2} \left( z - z_0 \right)^2} \sin \left( xy \right) \cos \left( z \right)
$$

In [1]:
function f(x0, y0, z0)
    
    result = 0.0
   
    for x=0:10, y=0:10, z=0:10
    
        result += exp(-(1/2)*(x - x0)^2)*exp(-(1/2)*(y - y0)^2)*exp(-(1/2)*(z - z0)^2)*sin(x*y)*cos(z)
        
    end
        
    return result
    
end

f (generic function with 1 method)

## Exact result

Let's first compute the exact value of the function for some values of $(x_0, y_0, z_0)$

In [6]:
@time f(2,6,10)

  0.000041 seconds


0.692056321795933

In [8]:
@time f(0,8,10)

  0.000039 seconds


-1.2701664008437514

In [9]:
@time f(7,2.4,3)

  0.000040 seconds


-1.0645141883730707

## Standard Monte Carlo

Let's now construct a standard Monte Carlo estimator for the above function

$$
\lim_{N\to\infty} f_{MC}(x_0, y_0, z_0, N) = f(x_0, y_0, z_0) 
$$

$$
f_{MC}(x_0, y_0, z_0, N) = \frac{V}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} f([ x, y, z ])
$$

where $ V = 11^3 $, as this is the total number of possible configurations

In [15]:
using Distributions, Random

In [46]:
function fMC(x0, y0, z0, N)
    
    draw = Array{Int}(undef, 3);
    draw_float_sample = Array{Float64}(undef, 1);
    Uniform_distribution = Uniform(0, 10);
    
    result = 0.0
   
    for i=1:N
        
          for i = 1:3
              rand!(Uniform_distribution, draw_float_sample)
              draw_float_sample[1] = round(Int64, draw_float_sample[1])
              draw[i] = Int(draw_float_sample[1])
          end
    
        result += exp(-(1/2)*(draw[1] - x0)^2)*exp(-(1/2)*(draw[2] - y0)^2)*exp(-(1/2)*(draw[3] - z0)^2)*sin(draw[1]*draw[2])*cos(draw[3])
        
    end
    
    result = result*11^3/N
        
    return result
    
end

fMC (generic function with 1 method)

In [85]:
@time fMC(2, 6, 10, 10^7)

  0.700662 seconds (2 allocations: 144 bytes)


0.6385288760178605

In [86]:
@time fMC(2, 6, 10, 10^7)

  0.713046 seconds (2 allocations: 144 bytes)


0.6606455083676828

In [87]:
@time fMC(2, 6, 10, 10^7)

  0.701753 seconds (2 allocations: 144 bytes)


0.6628532618085105

In [88]:
@time fMC(2, 6, 10, 10^7)

  0.707672 seconds (2 allocations: 144 bytes)


0.6448320741738877

$\textbf{The converge is slow}$: we need $10^7$ iterations to see a result stable up to the first significant digit. 

More importantly, $\textbf{the converges is biased}$, since the sum is highly oscillating and has several cancellations due to the negative terms.

## VEGAS